In [1]:
import pandas as pd
import os
import requests
import json
import time
import re

In [2]:
def get_token():
    with open("token.json", "r") as f:
        data = json.load(f)
    return data["token"]

In [3]:
TOKEN = get_token()

In [4]:
def make_request(url, headers):
    cnt = 0
    while cnt <= 5:
        try:
            response = requests.get(url, headers=headers)
            if response.status_code == requests.codes.ok:
                return response.text
        except requests.exceptions.RequestException as e:
            time.sleep(0.5)
            cnt += 1
    raise Exception("request failed!")

In [5]:
def get_tmdb_data(imdbId):
    print(imdbId)
    headers = {
        "accept": "application/json",
        "Authorization": TOKEN
    }
    data_raw = make_request(f"https://api.themoviedb.org/3/find/{imdbId}?external_source=imdb_id", headers)
    data = json.loads(data_raw)
    try:
        tmdbId = data['movie_results'][0]['id']
    except:
        return (0, 0, 0, 0, 0, 0, 0, 0)
    
    data_raw = make_request(f"https://api.themoviedb.org/3/movie/{tmdbId}?language=en-US", headers) 
    data = json.loads(data_raw)
    try:
        print(data['original_title'])
        
        productionCountries = ",".join([country['name'] for country in data["production_countries"]])
        studios = ",".join([company['name'] for company in data['production_companies']])
        originalLanguage = data['original_language']
        isAdult = data['adult']
        budget = data['budget']
        revenue = data['revenue']
        voteAverage = data['vote_average']
        voteCount = data['vote_count']
    except:
        return (0, 0, 0, 0, 0, 0, 0, 0)
    
    return productionCountries, studios, originalLanguage, isAdult, budget, revenue, voteAverage, voteCount

In [6]:
def get_letterboxd_data(imdbId):
    print(imdbId)
    headers = {
        "accept": "application/json"
    }
    data_raw = make_request(f"https://letterboxd.com/imdb/{imdbId}", headers)
    
    json_pattern = r'(\{.*?"productionCompany".*?"ratingValue".*?\}\})'
    json_match = re.search(json_pattern, data_raw)
    
    if json_match:
        try:
            json_data = json.loads(json_match.group(1))

            rating = json_data.get("aggregateRating", {}).get("ratingValue")
            numVotes = json_data.get("aggregateRating", {}).get("ratingCount")
            directors = ",".join([director['name'] for director in json_data.get("director")])
            
            return directors, rating, numVotes
        except:
            print(f"json loading failed")
            return (0, 0, 0)
    return (0, 0, 0)

In [7]:
get_tmdb_data("tt0152267")

tt0152267


(0, 0, 0, 0, 0, 0, 0, 0)

In [8]:
get_letterboxd_data("tt3907674")

tt3907674


(0, 0, 0)

# IMDb

In [9]:
imdb_ratings = pd.read_csv(f'raw/title.ratings.tsv', sep='\t', na_values="\\N")
imdb_titles = pd.read_csv(f'raw/title.basics.tsv', sep='\t', na_values="\\N")

imdb_merged = pd.merge(
    imdb_titles[imdb_titles['titleType'] == 'movie'],
    imdb_ratings,
    on='tconst',
).fillna({'averageRating': -1, 'numVotes': -1, 'runtimeMinutes': -1, 'genres': ''})

C:\Users\danpf\AppData\Local\Temp\ipykernel_17588\2287869136.py:2: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  imdb_titles = pd.read_csv(f'raw/title.basics.tsv', sep='\t', na_values="\\N")


In [10]:
imdb_merged = imdb_merged[
    (imdb_merged['averageRating'] != -1) &
    (imdb_merged["numVotes"] != -1) &
    (imdb_merged["runtimeMinutes"] != 1)
]

print(f"imdb titles total: {imdb_ratings.shape[0]}")
print(f"imdb movies with complete information (rating and runtime): {imdb_merged.shape[0]}")

imdb titles total: 1530505
imdb movies with complete information (rating and runtime): 324690


In [11]:
imdb_merged['startYear'] = imdb_merged['startYear'].astype(str)
imdb_merged['startYear'] = imdb_merged['startYear'].str[:4]

In [12]:
imdb_merged = imdb_merged.rename(columns={"tconst": "imdbId", "averageRating": "imdbScore", "numVotes": "imdbNumVotes"})

In [13]:
imdb_merged.head()

,imdbId,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres,imdbScore,imdbNumVotes
0,tt0000009,movie,Miss Jerry,Miss Jerry,0.0,1894,NaN,45.0,Romance,5.4,218
1,tt0000147,movie,The Corbett-Fitzsimmons Fight,The Corbett-Fitzsimmons Fight,0.0,1897,NaN,100.0,"Documentary,News,Sport",5.3,549
2,tt0000502,movie,Bohemios,Bohemios,0.0,1905,NaN,100.0,,3.8,20
3,tt0000574,movie,The Story of the Kelly Gang,The Story of the Kelly Gang,0.0,1906,NaN,70.0,"Action,Adventure,Biography",6.0,969
4,tt0000591,movie,The Prodigal Son,L'enfant prodigue,0.0,1907,NaN,90.0,Drama,5.6,30


# Rotten Tomatoes

In [14]:
reviews_df = pd.read_csv(f"raw/rotten_tomatoes_movie_reviews.csv")
movies_df = pd.read_csv(f"raw/rotten_tomatoes_movies.csv")

tomatoes_merged = pd.merge(
    movies_df[['id', 'title', 'audienceScore', 'tomatoMeter', 'releaseDateTheaters']],
    reviews_df[['id', 'creationDate', 'reviewState']],
    on='id'
).fillna({'tomatoMeter': -1, 'audienceScore': -1, 'releaseDateTheaters': -1})

In [15]:
tomatoes_merged['releaseYear'] = tomatoes_merged['releaseDateTheaters'].str[:4]

In [16]:
tomatoes_merged = tomatoes_merged[
    (tomatoes_merged['tomatoMeter'] != -1) &
    (tomatoes_merged['audienceScore'] != -1) &
    (tomatoes_merged['releaseDateTheaters'] != -1)
]

In [17]:
print(f"rotten tomatoes movies: {movies_df.shape[0]}")
print(f"rotten tomatoes movies with ratings: {len(tomatoes_merged['id'].unique())}")

rotten tomatoes movies: 143258
rotten tomatoes movies with ratings: 18507


In [18]:
# Convert reviewState to numerical values
tomatoes_merged["score"] = tomatoes_merged["reviewState"].map({"fresh": 10, "rotten": 0})

# Extract year from creationDate
tomatoes_merged["year"] = pd.to_datetime(tomatoes_merged["creationDate"]).dt.year

# Group by id and year, compute the average score and count reviews
yearly_data = tomatoes_merged.groupby(["id", "year"]).agg(
    avg_score=("score", "mean"),
    num_reviews=("score", "count")
).reset_index()

# Round scores and convert to integers
yearly_data["avg_score"] = yearly_data["avg_score"].round().astype(int)

# Convert to dictionary format
score_dicts = yearly_data.groupby("id").apply(
    lambda x: json.dumps(dict(zip(x["year"], x["avg_score"])), separators=(",", ":"))
).reset_index()

review_count_dicts = yearly_data.groupby("id").apply(
    lambda x: json.dumps(dict(zip(x["year"], x["num_reviews"])), separators=(",", ":"))
).reset_index()

# Compute total number of reviews for each movie
total_reviews = tomatoes_merged.groupby("id")["score"].count().reset_index()
total_reviews.rename(columns={"score": "rottenTomatoesNumVotes"}, inplace=True)

# Merge with unique movie data
unique_movies = tomatoes_merged.drop_duplicates(subset=["id"])[
    ["id", "title", "audienceScore", "tomatoMeter", "releaseDateTheaters", "releaseYear"]
]

tomatoes_new = unique_movies.merge(score_dicts, on="id").merge(review_count_dicts, on="id").merge(total_reviews, on="id")
tomatoes_new.rename(columns={0: "yearly_scores", 1: "yearlyNumReviews"}, inplace=True)

C:\Users\danpf\AppData\Local\Temp\ipykernel_17588\4215766069.py:17: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  score_dicts = yearly_data.groupby("id").apply(
C:\Users\danpf\AppData\Local\Temp\ipykernel_17588\4215766069.py:21: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  review_count_dicts = yearly_data.groupby("id").apply(


In [19]:
tomatoes_new = tomatoes_new.rename(columns={"0_x": "rottenTomatoesReviews", "0_y": "rottenTomatoesReviewsNumVotes"})

In [20]:
tomatoes_new.head()

,id,title,audienceScore,tomatoMeter,releaseDateTheaters,releaseYear,rottenTomatoesReviews,rottenTomatoesReviewsNumVotes,rottenTomatoesNumVotes
0,adrift_2018,Adrift,65.0,69.0,2018-06-01,2018,"{""2018"":0,""2019"":7,""2020"":5,""2021"":5,""2022"":10}","{""2018"":4,""2019"":19,""2020"":11,""2021"":4,""2022"":2}",40
1,1035316-born_to_kill,Born to Kill,74.0,83.0,1947-04-30,1947,"{""2003"":0,""2005"":10,""2006"":10,""2016"":10,""2020""...","{""2003"":1,""2005"":1,""2006"":2,""2016"":1,""2020"":1}",6
2,1221483-paa,Paa,67.0,50.0,2009-12-04,2009,"{""2009"":6,""2017"":0,""2019"":0,""2020"":10}","{""2009"":5,""2017"":1,""2019"":1,""2020"":1}",8
3,sarah_palin_you_betcha,Sarah Palin: You Betcha!,61.0,32.0,2011-09-30,2011,"{""2011"":3}","{""2011"":31}",31
4,a_state_of_mind_2005,A State of Mind,92.0,89.0,2005-08-10,2005,"{""2005"":10,""2006"":10,""2007"":3}","{""2005"":22,""2006"":2,""2007"":3}",27


In [21]:
tomatoes_new.shape[0]

18507

In [22]:
filtered_df = pd.merge(
    imdb_merged,
    tomatoes_new,
    left_on=['primaryTitle', 'startYear'],
    right_on=['title', 'releaseYear'],
    how='inner'
)

In [23]:
filtered_df = filtered_df.drop(columns=['primaryTitle', 'releaseYear', 'titleType', 'isAdult', 'endYear', 'id', 'originalTitle'])

In [24]:
filtered_df = filtered_df.rename(columns={"tconst": "imdbId"})

In [25]:
print(f"number of movies from rotten tomatoes and IMDb merge data: {filtered_df.shape[0]}")

number of movies from rotten tomatoes and IMDb merge data: 12268


In [26]:
#filtered_df = filtered_df[0:10]

In [27]:
filtered_df.head()

,imdbId,startYear,runtimeMinutes,genres,imdbScore,imdbNumVotes,title,audienceScore,tomatoMeter,releaseDateTheaters,rottenTomatoesReviews,rottenTomatoesReviewsNumVotes,rottenTomatoesNumVotes
0,tt0004707,1914,82.0,Comedy,6.2,3791,Tillie's Punctured Romance,42.0,90.0,1914-11-14,"{""2007"":10,""2008"":10,""2009"":5,""2010"":10,""2020""...","{""2007"":2,""2008"":1,""2009"":2,""2010"":1,""2020"":3,...",10
1,tt0004972,1915,195.0,"Drama,War",6.1,27113,The Birth of a Nation,54.0,91.0,1915-03-03,"{""2000"":10,""2001"":10,""2002"":10,""2003"":8,""2004""...","{""2000"":1,""2001"":2,""2002"":6,""2003"":5,""2004"":5,...",42
2,tt0005078,1915,59.0,"Drama,Romance",6.5,2913,The Cheat,47.0,90.0,1915-12-12,"{""2004"":10,""2005"":10,""2006"":0,""2008"":10,""2019""...","{""2004"":1,""2005"":2,""2006"":1,""2008"":1,""2019"":4,...",10
3,tt0005960,1915,72.0,"Biography,Crime,Drama",6.8,1484,The Regeneration,69.0,100.0,1915-09-13,"{""2005"":10,""2013"":10}","{""2005"":1,""2013"":2}",3
4,tt0006864,1916,163.0,"Drama,History",7.7,17240,Intolerance,77.0,97.0,1916-09-05,"{""2000"":10,""2002"":7,""2003"":10,""2004"":10,""2005""...","{""2000"":2,""2002"":3,""2003"":4,""2004"":6,""2005"":2,...",38


In [28]:
filtered_df.to_csv("raw/temp.csv", index=False)

# TMDb

In [29]:
if os.path.isfile("processed/tmdb.csv"):
    print("found existing csv file")
    tmdb = pd.read_csv("processed/tmdb.csv", index_col=[0])
else:
    tmdb = filtered_df[['imdbId']]

    print(f"fetching api data for {tmdb.shape[0]} movies")
    tmdb['raw'] = tmdb['imdbId'].apply(get_tmdb_data)

    tmdb['productionCountries'], tmdb['studios'], tmdb['originalLanguage'], tmdb['isAdult'], tmdb['budget'], tmdb['revenue'], tmdb['tmdbAverageScore'], tmdb['tmdbNumVotes'] = zip(*tmdb.raw)
    tmdb = tmdb.drop(columns=['raw'])

    tmdb = tmdb[
        (tmdb['budget'] != 0) &
        (tmdb['revenue'] != 0)
    ]

    print(f"tmdb shape: {tmdb.shape}")
    tmdb.to_csv("processed/tmdb.csv", index=False)

found existing csv file


In [30]:
tmdb.head()

,productionCountries,studios,originalLanguage,isAdult,budget,revenue,tmdbAverageScore,tmdbNumVotes
imdbId,,,,,,,,
tt0004972,United States of America,Epoch Film Co.,en,False,100000,11000000,6.0,542
tt0005078,United States of America,"Jesse L. Lasky Feature Play Company,Paramount ...",en,False,17311,137365,5.9,64
tt0006864,United States of America,"Triangle Film Corporation,Wark Producing Corp.",en,False,385907,1750000,7.1,341
tt0012349,United States of America,Charles Chaplin Productions,en,False,250000,2500000,8.2,2139
tt0013140,United States of America,Universal Film Manufacturing Company,en,False,1100000,400200,6.6,90


In [31]:
print(f"number of movies with information about budget and revenue from TMDb: {tmdb.shape[0]}")

number of movies with information about budget and revenue from TMDb: 5617


In [32]:
third_df = pd.merge(
    filtered_df,
    tmdb,
    how="inner",
    on="imdbId",
)

# LETTERBOXD

In [33]:
if os.path.isfile("processed/letterboxd.csv"):
    letterboxd = pd.read_csv("processed/letterboxd.csv", index_col=[0])
else:
    letterboxd = filtered_df[['imdbId']]

    print(f"scraping letterboxd for {tmdb.shape[0]} movies")
    letterboxd['raw'] = letterboxd['imdbId'].apply(get_letterboxd_data)

    letterboxd['directors'], letterboxd['letterboxdAverageScore'], letterboxd['letterboxdNumVotes'] = zip(*letterboxd.raw)
    letterboxd = letterboxd.drop(columns=['raw'])
    
    letterboxd = letterboxd[
        (letterboxd['letterboxdAverageScore'] != 0) &
        (letterboxd['letterboxdNumVotes'] != 0)
    ]

    print(f"letterboxd shape: {letterboxd.shape}")
    letterboxd.to_csv("processed/letterboxd.csv", index=False)

scraping letterboxd for 5617 movies
tt0004707
tt0004972
tt0005078
tt0005960
tt0006864
tt0007340
tt0009682
tt0009937
tt0009968
tt0010806
tt0011130
tt0011387
tt0011565
tt0011652
tt0011841
tt0011870
tt0011979
tt0012349
tt0012532
tt0012675
tt0012752
tt0012938
tt0012952
tt0013140
tt0013201
tt0013427
tt0013597
tt0014142
tt0014341
tt0014532
tt0014672
tt0015016
tt0015064
tt0015119
tt0015163
tt0015361
tt0015400
tt0015532
tt0015624
tt0015634
tt0015841
tt0015863
tt0015864
tt0015873
tt0015881
tt0016004
tt0016039
tt0016220
tt0016308
tt0016332
tt0016473
tt0016847
tt0016884
tt0017136
tt0017162
tt0017739
tt0017765
tt0017843
tt0017918
tt0018033
tt0018037
tt0018051
tt0018054
tt0018107
tt0018328
tt0018455
tt0018526
tt0018578
tt0018737
tt0018742
tt0018773
tt0018806
tt0019071
tt0019130
tt0019195
tt0019254
tt0019304
tt0019344
tt0019379
tt0019412
tt0019421
tt0019553
tt0019558
tt0019585
tt0019630
tt0019649
tt0019702
tt0019729
tt0019788
tt0019823
tt0019946
tt0019959
tt0020030
tt0020112
tt0020269
tt0020442
tt00

C:\Users\danpf\AppData\Local\Temp\ipykernel_17588\1533592817.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  letterboxd['raw'] = letterboxd['imdbId'].apply(get_letterboxd_data)
C:\Users\danpf\AppData\Local\Temp\ipykernel_17588\1533592817.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  letterboxd['directors'], letterboxd['letterboxdAverageScore'], letterboxd['letterboxdNumVotes'] = zip(*letterboxd.raw)
C:\Users\danpf\AppData\Local\Temp\ipykernel_17588\1533592817.py:9: SettingWithCopyWarning: 
A value

In [34]:
fourth_df = pd.merge(
    third_df,
    letterboxd,
    how="inner",
    on="imdbId",
)

In [35]:
print(f"final shape: {fourth_df.shape}")
fourth_df.head()

final shape: (5574, 24)


,imdbId,startYear,runtimeMinutes,genres,imdbScore,imdbNumVotes,title,audienceScore,tomatoMeter,releaseDateTheaters,...,studios,originalLanguage,isAdult,budget,revenue,tmdbAverageScore,tmdbNumVotes,directors,letterboxdAverageScore,letterboxdNumVotes
0,tt0004972,1915,195.0,"Drama,War",6.1,27113,The Birth of a Nation,54.0,91.0,1915-03-03,...,Epoch Film Co.,en,False,100000,11000000,6.0,542,D.W. Griffith,2.31,32086
1,tt0005078,1915,59.0,"Drama,Romance",6.5,2913,The Cheat,47.0,90.0,1915-12-12,...,"Jesse L. Lasky Feature Play Company,Paramount ...",en,False,17311,137365,5.9,64,Cecil B. DeMille,2.99,3745
2,tt0006864,1916,163.0,"Drama,History",7.7,17240,Intolerance,77.0,97.0,1916-09-05,...,"Triangle Film Corporation,Wark Producing Corp.",en,False,385907,1750000,7.1,341,D.W. Griffith,3.69,18974
3,tt0012349,1921,68.0,"Comedy,Drama,Family",8.2,139644,The Kid,95.0,100.0,1921-01-21,...,Charles Chaplin Productions,en,False,250000,2500000,8.2,2139,Charlie Chaplin,4.21,102205
4,tt0013140,1922,117.0,"Drama,Thriller",7.0,4163,Foolish Wives,70.0,89.0,1922-01-11,...,Universal Film Manufacturing Company,en,False,1100000,400200,6.6,90,Erich von Stroheim,3.67,4822


In [36]:
fourth_df.to_csv('processed/movies.csv')